In [1]:
#load core libraries
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.ops.gen_dataset_ops import map_dataset

#load the data once at the beginning to save time
df = pd.read_parquet("hf://datasets/pcuenq/oxford-pets/data/train-00000-of-00001-ecc2afb43dedd5e0.parquet")

In [2]:
#get a sense of the data structure
df.head()

,path,label,dog,image
0,/data/datasets/magic-ml/oxford-iiit-pet/images...,Siamese,False,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1,/data/datasets/magic-ml/oxford-iiit-pet/images...,Birman,False,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2,/data/datasets/magic-ml/oxford-iiit-pet/images...,shiba inu,True,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3,/data/datasets/magic-ml/oxford-iiit-pet/images...,staffordshire bull terrier,True,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4,/data/datasets/magic-ml/oxford-iiit-pet/images...,basset hound,True,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...


In [4]:
#making a python list from the series (column) in df
images = [img['bytes'] for img in df["image"]]
is_dog = df['dog'].astype(int).values

IMG_SIZE = (224,224)

def preprocess(image, label):
    image_new = tf.image.decode_jpeg(image, channels=3)
    #resize the image for the models input
    image_new = tf.image.resize(image_new, IMG_SIZE)
    #resize the image to be within the confines of tensors [-1,1]
    image_new = image_new / 255.0
    return image_new, label

dataset = tf.data.Dataset.from_tensor_slices((images, is_dog))
dataset = dataset.map(preprocess)

print(dataset)

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [5]:
import keras
training_size = 0.8
#test size is not needed as we will take whatever is leftover after the split

#split the dataset into subsets for training purposes
split_tuple = tf.keras.utils.split_dataset(dataset = dataset, left_size = training_size, shuffle=True, seed=0)

#having an error on line 6:
#Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

val_test_split = tf.keras.utils.split_dataset(dataset = split_tuple[0], left_size = 0.875, shuffle=False)
train_dataset = val_test_split[0]
validation_dataset = val_test_split[1]
test_dataset = split_tuple[1]

Everything Below is untested and in the works still

In [6]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [7]:
# DONT RUN FOR NOW
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

SyntaxError: expected '(' (3985758228.py, line 1)

In [8]:
#load the base model
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [12]:
base_model.trainable = False
inputs = keras.Input(shape=IMG_SHAPE)
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary(show_trainable=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_3 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenetv2_1.00_224        │ (None, 7, 7, 1280)    │  2,257,984 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 1280)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 1)             │      1,281 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [10]:
from tensorflow import data as tf_data

batch_size = 64

train_dataset = train_dataset.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
validation_dataset = validation_dataset.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_dataset = test_dataset.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [14]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 2
print("Fitting the top layer of the model")
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)

Fitting the top layer of the model
Epoch 1/2
81/81 ━━━━━━━━━━━━━━━━━━━━ 79s 921ms/step - binary_accuracy: 0.9884 - loss: 0.0419 - val_binary_accuracy: 0.9882 - val_loss: 0.0384
Epoch 2/2
81/81 ━━━━━━━━━━━━━━━━━━━━ 73s 905ms/step - binary_accuracy: 0.9906 - loss: 0.0283 - val_binary_accuracy: 0.9916 - val_loss: 0.0332


In [20]:
print(test_dataset)

<CacheDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [21]:
for images, labels in test_dataset.take(1):  # Limit to one batch for inspection
    print("Labels (tf.int64 array):", labels.numpy())

Labels (tf.int64 array): [1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1]


In [23]:
print("Test dataset evaluation, pre-fine tuning")
model.evaluate(test_dataset)
model.evaluate(train_dataset)

Test dataset evaluation
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 519ms/step - binary_accuracy: 1.0000 - loss: 0.0237
81/81 ━━━━━━━━━━━━━━━━━━━━ 65s 809ms/step - binary_accuracy: 0.9933 - loss: 0.0203


[0.01919730193912983, 0.9955538511276245]

In [25]:
base_model.trainable = True
model.summary(show_trainable=True)

model.compile(
    optimizer=keras.optimizers.Adam(1e-7),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 1
print("Fitting the end-to-end model")
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_3 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenetv2_1.00_224        │ (None, 7, 7, 1280)    │  2,257,984 │   Y   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 1280)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 1)             │      1,281 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 2,225,153 (8.49 MB)

 Non-trainable params: 34,112 (133.25 KB)

Fitting the end-to-end model
81/81 ━━━━━━━━━━━━━━━━━━━━ 485s 6s/step - binary_accuracy: 0.9216 - loss: 0.2500 - val_binary_accuracy: 0.9747 - val_loss: 0.1029


In [26]:
print("Test dataset evaluation")
model.evaluate(test_dataset)
model.evaluate(train_dataset)

Test dataset evaluation
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 638ms/step - binary_accuracy: 0.9815 - loss: 0.0699
81/81 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - binary_accuracy: 0.9880 - loss: 0.0423


[0.04731040820479393, 0.9858882427215576]